<a href="https://colab.research.google.com/github/SrijaG29/Laptop_price_prediction_using_pyspark/blob/main/laptop_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'laptop-price-prediction-cleaned-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3408087%2F5937858%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240827%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240827T075642Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5fcd48bdcbd664161970eb37632a6f23539772763f0bf648a40c784711a6361d4a40bf99092e3afb72a1c1a2888b6f73b430734c3023a3ac4a432024bd434c3b43ca916b55cf560c7847d0dfce0fc3d918628ed646e95d1a153658b4fd4641de171796b065c4555205dfbb8ef21d84af0a361a38de986eb5771c7c491f55f03eb730524f0e23236c4c8d3955e282906fc8ae65f0b08a64e4d8c0cea4cdd05da42c40bdc90e63d955bb6f7bee8b4b0c02aae0130c88310001fb1495c0fad98d53b0232a3cdf1f34daf029fb36581cf1f8f4fef0efb391116fc2adf17f91735d78f981f33fbfbfb4e0eda49e5106289c77dee2e78f1c05e37d0c680fb60bdb8b28'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 22067 bytes downloaded
Downloaded and uncompressed: laptop-price-prediction-cleaned-dataset
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/laptop-price-prediction-cleaned-dataset/laptop_data_cleaned.csv


In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=b28bf1c34905803d2057b837ab4ea087f5b5bbad62f14dc5fdfb1fb8bb0f8961
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [6]:
from pyspark.sql import SparkSession

In [7]:
spark =(
    SparkSession
    .builder
    .appName('Laptop_price_prediction')
    .master("local[*]")
    .getOrCreate()
)

In [8]:
spark

In [24]:
from pyspark.sql.types import StructField,StructType,StringType,IntegerType,FloatType
schema = (
    StructType(
        [
            StructField('Company',StringType(),True),
            StructField('TypeName',StringType(),True),
            StructField('Ram',IntegerType(),True),
            StructField('Weight',FloatType(),True),
            StructField('Price',FloatType(),True),
            StructField('TouchScreen',IntegerType(),True),
            StructField('Ips',IntegerType(),True),
            StructField('Ppi',FloatType(),True),
            StructField('Cpu_brand',StringType(),True),
            StructField('HDD',IntegerType(),True),
            StructField('SSD',IntegerType(),True),
            StructField('Gpu_brand',StringType(),True),
            StructField('Os',StringType(),True)
        ]
    )
)

In [48]:
laptop_details = spark.read.format('csv').option('header',True).schema(schema).load("/kaggle/input/laptop-price-prediction-cleaned-dataset/laptop_data_cleaned.csv")

In [49]:
laptop_details.show(truncate = False)

+-------+---------+---+------+---------+-----------+---+----------+---------------------+----+---+---------+-------+
|Company|TypeName |Ram|Weight|Price    |TouchScreen|Ips|Ppi       |Cpu_brand            |HDD |SSD|Gpu_brand|Os     |
+-------+---------+---+------+---------+-----------+---+----------+---------------------+----+---+---------+-------+
|Apple  |Ultrabook|8  |1.37  |11.175755|0          |1  |226.983   |Intel Core i5        |0   |128|Intel    |Mac    |
|Apple  |Ultrabook|8  |1.34  |10.776777|0          |0  |127.67794 |Intel Core i5        |0   |0  |Intel    |Mac    |
|HP     |Notebook |8  |1.86  |10.329931|0          |0  |141.212   |Intel Core i5        |0   |256|Intel    |Others |
|Apple  |Ultrabook|16 |1.83  |11.814476|0          |1  |220.53462 |Intel Core i7        |0   |512|AMD      |Mac    |
|Apple  |Ultrabook|8  |1.37  |11.473101|0          |1  |226.983   |Intel Core i5        |0   |256|Intel    |Mac    |
|Acer   |Notebook |4  |2.1   |9.967026 |0          |0  |100.4546

In [50]:
laptop_details.printSchema()

root
 |-- Company: string (nullable = true)
 |-- TypeName: string (nullable = true)
 |-- Ram: integer (nullable = true)
 |-- Weight: float (nullable = true)
 |-- Price: float (nullable = true)
 |-- TouchScreen: integer (nullable = true)
 |-- Ips: integer (nullable = true)
 |-- Ppi: float (nullable = true)
 |-- Cpu_brand: string (nullable = true)
 |-- HDD: integer (nullable = true)
 |-- SSD: integer (nullable = true)
 |-- Gpu_brand: string (nullable = true)
 |-- Os: string (nullable = true)



In this dataset we don't require TypeName.
Because we decide the price of the latop based on the features but not on Typename.

In [51]:
laptop_details = laptop_details.drop('TypeName')
laptop_details.show()

+-------+---+------+---------+-----------+---+----------+--------------------+----+---+---------+-------+
|Company|Ram|Weight|    Price|TouchScreen|Ips|       Ppi|           Cpu_brand| HDD|SSD|Gpu_brand|     Os|
+-------+---+------+---------+-----------+---+----------+--------------------+----+---+---------+-------+
|  Apple|  8|  1.37|11.175755|          0|  1|   226.983|       Intel Core i5|   0|128|    Intel|    Mac|
|  Apple|  8|  1.34|10.776777|          0|  0| 127.67794|       Intel Core i5|   0|  0|    Intel|    Mac|
|     HP|  8|  1.86|10.329931|          0|  0|   141.212|       Intel Core i5|   0|256|    Intel| Others|
|  Apple| 16|  1.83|11.814476|          0|  1| 220.53462|       Intel Core i7|   0|512|      AMD|    Mac|
|  Apple|  8|  1.37|11.473101|          0|  1|   226.983|       Intel Core i5|   0|256|    Intel|    Mac|
|   Acer|  4|   2.1| 9.967026|          0|  0|100.454666|       AMD Processor| 500|  0|      AMD|Windows|
|  Apple| 16|  2.04|11.644108|          0|  1|

In [52]:
import pandas as pd

laptop_details_pd = laptop_details.toPandas()

laptop_details_pd.isna().sum()


,0
Company,0
Ram,0
Weight,0
Price,0
TouchScreen,0
Ips,0
Ppi,0
Cpu_brand,0
HDD,0
SSD,0


In [53]:
print(laptop_details.dtypes)

[('Company', 'string'), ('Ram', 'int'), ('Weight', 'float'), ('Price', 'float'), ('TouchScreen', 'int'), ('Ips', 'int'), ('Ppi', 'float'), ('Cpu_brand', 'string'), ('HDD', 'int'), ('SSD', 'int'), ('Gpu_brand', 'string'), ('Os', 'string')]


In [54]:
from pyspark.sql.functions import col,round
laptop_details = laptop_details.withColumn('Weight',round(col('Weight'),0).cast('int'))\
                 .withColumn('Ram',round(col('Ram'),0).cast('int'))\
                 .withColumn('Ppi',round(col('Ppi'),0).cast('int'))\
                 .withColumn('Price',round(col('Price'),0).cast('int'))

laptop_details.show(truncate = False)


+-------+---+------+-----+-----------+---+---+---------------------+----+---+---------+-------+
|Company|Ram|Weight|Price|TouchScreen|Ips|Ppi|Cpu_brand            |HDD |SSD|Gpu_brand|Os     |
+-------+---+------+-----+-----------+---+---+---------------------+----+---+---------+-------+
|Apple  |8  |1     |11   |0          |1  |227|Intel Core i5        |0   |128|Intel    |Mac    |
|Apple  |8  |1     |11   |0          |0  |128|Intel Core i5        |0   |0  |Intel    |Mac    |
|HP     |8  |2     |10   |0          |0  |141|Intel Core i5        |0   |256|Intel    |Others |
|Apple  |16 |2     |12   |0          |1  |221|Intel Core i7        |0   |512|AMD      |Mac    |
|Apple  |8  |1     |11   |0          |1  |227|Intel Core i5        |0   |256|Intel    |Mac    |
|Acer   |4  |2     |10   |0          |0  |100|AMD Processor        |500 |0  |AMD      |Windows|
|Apple  |16 |2     |12   |0          |1  |221|Intel Core i7        |0   |0  |Intel    |Mac    |
|Apple  |8  |1     |11   |0          |0 

In [55]:
laptop_details.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Ram: integer (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- Price: integer (nullable = true)
 |-- TouchScreen: integer (nullable = true)
 |-- Ips: integer (nullable = true)
 |-- Ppi: integer (nullable = true)
 |-- Cpu_brand: string (nullable = true)
 |-- HDD: integer (nullable = true)
 |-- SSD: integer (nullable = true)
 |-- Gpu_brand: string (nullable = true)
 |-- Os: string (nullable = true)



In [56]:
laptop_details.groupBy('Company').count().show() # 19 unique values

+---------+-----+
|  Company|count|
+---------+-----+
|    Razer|    7|
|  Fujitsu|    3|
|   Huawei|    2|
|   Xiaomi|    4|
|       HP|  268|
|     Dell|  291|
|     Vero|    4|
|     Acer|  101|
|     Asus|  151|
|   Lenovo|  289|
| Mediacom|    7|
|  Samsung|    8|
|   Google|    3|
|       LG|    3|
|    Chuwi|    3|
|Microsoft|    6|
|    Apple|   21|
|      MSI|   54|
|  Toshiba|   48|
+---------+-----+



In [57]:
laptop_details.groupBy('Cpu_brand').count().show() # 5 unique values

+--------------------+-----+
|           Cpu_brand|count|
+--------------------+-----+
|       Intel Core i3|  134|
|Other Intel Proce...|  141|
|       Intel Core i5|  423|
|       Intel Core i7|  515|
|       AMD Processor|   60|
+--------------------+-----+



In [58]:
laptop_details.groupBy('Gpu_brand').count().show() # 3 unique values

+---------+-----+
|Gpu_brand|count|
+---------+-----+
|      AMD|  174|
|    Intel|  703|
|   Nvidia|  396|
+---------+-----+



In [59]:
laptop_details.groupBy('Os').count().show() # 3 unique values

+-------+-----+
|     Os|count|
+-------+-----+
| Others|  152|
|    Mac|   21|
|Windows| 1100|
+-------+-----+



In [67]:
laptop_details_copy = laptop_details.select('*')
laptop_details_copy.show()

+-------+---+------+-----+-----------+---+---+--------------------+----+---+---------+-------+
|Company|Ram|Weight|Price|TouchScreen|Ips|Ppi|           Cpu_brand| HDD|SSD|Gpu_brand|     Os|
+-------+---+------+-----+-----------+---+---+--------------------+----+---+---------+-------+
|  Apple|  8|     1|   11|          0|  1|227|       Intel Core i5|   0|128|    Intel|    Mac|
|  Apple|  8|     1|   11|          0|  0|128|       Intel Core i5|   0|  0|    Intel|    Mac|
|     HP|  8|     2|   10|          0|  0|141|       Intel Core i5|   0|256|    Intel| Others|
|  Apple| 16|     2|   12|          0|  1|221|       Intel Core i7|   0|512|      AMD|    Mac|
|  Apple|  8|     1|   11|          0|  1|227|       Intel Core i5|   0|256|    Intel|    Mac|
|   Acer|  4|     2|   10|          0|  0|100|       AMD Processor| 500|  0|      AMD|Windows|
|  Apple| 16|     2|   12|          0|  1|221|       Intel Core i7|   0|  0|    Intel|    Mac|
|  Apple|  8|     1|   11|          0|  0|128|    

In [68]:
from pyspark.ml.feature import StringIndexer

# Initialize the StringIndexer
indexer = StringIndexer(inputCol="Company", outputCol="Company_encoded")

# Fit the indexer to the DataFrame and transform it
laptop_details_copy = indexer.fit(laptop_details_copy).transform(laptop_details_copy)

In [69]:
from pyspark.ml.feature import StringIndexer

# Initialize the StringIndexer
indexer = StringIndexer(inputCol="Cpu_brand", outputCol="Cpu_brand_encoded")

# Fit the indexer to the DataFrame and transform it
laptop_details_copy = indexer.fit(laptop_details_copy).transform(laptop_details_copy)

In [70]:
from pyspark.ml.feature import StringIndexer

# Initialize the StringIndexer
indexer = StringIndexer(inputCol="Gpu_brand", outputCol="Gpu_brand_encoded")

# Fit the indexer to the DataFrame and transform it
laptop_details_copy = indexer.fit(laptop_details_copy).transform(laptop_details_copy)

In [71]:
from pyspark.ml.feature import StringIndexer

# Initialize the StringIndexer
indexer = StringIndexer(inputCol="Os", outputCol="Os_encoded")

# Fit the indexer to the DataFrame and transform it
laptop_details_copy = indexer.fit(laptop_details_copy).transform(laptop_details_copy)

In [72]:
laptop_details_copy.show()

+-------+---+------+-----+-----------+---+---+--------------------+----+---+---------+-------+---------------+-----------------+-----------------+----------+
|Company|Ram|Weight|Price|TouchScreen|Ips|Ppi|           Cpu_brand| HDD|SSD|Gpu_brand|     Os|Company_encoded|Cpu_brand_encoded|Gpu_brand_encoded|Os_encoded|
+-------+---+------+-----+-----------+---+---+--------------------+----+---+---------+-------+---------------+-----------------+-----------------+----------+
|  Apple|  8|     1|   11|          0|  1|227|       Intel Core i5|   0|128|    Intel|    Mac|            7.0|              1.0|              0.0|       2.0|
|  Apple|  8|     1|   11|          0|  0|128|       Intel Core i5|   0|  0|    Intel|    Mac|            7.0|              1.0|              0.0|       2.0|
|     HP|  8|     2|   10|          0|  0|141|       Intel Core i5|   0|256|    Intel| Others|            2.0|              1.0|              0.0|       1.0|
|  Apple| 16|     2|   12|          0|  1|221|      

In [73]:
laptop_details_copy.show()

+-------+---+------+-----+-----------+---+---+--------------------+----+---+---------+-------+---------------+-----------------+-----------------+----------+
|Company|Ram|Weight|Price|TouchScreen|Ips|Ppi|           Cpu_brand| HDD|SSD|Gpu_brand|     Os|Company_encoded|Cpu_brand_encoded|Gpu_brand_encoded|Os_encoded|
+-------+---+------+-----+-----------+---+---+--------------------+----+---+---------+-------+---------------+-----------------+-----------------+----------+
|  Apple|  8|     1|   11|          0|  1|227|       Intel Core i5|   0|128|    Intel|    Mac|            7.0|              1.0|              0.0|       2.0|
|  Apple|  8|     1|   11|          0|  0|128|       Intel Core i5|   0|  0|    Intel|    Mac|            7.0|              1.0|              0.0|       2.0|
|     HP|  8|     2|   10|          0|  0|141|       Intel Core i5|   0|256|    Intel| Others|            2.0|              1.0|              0.0|       1.0|
|  Apple| 16|     2|   12|          0|  1|221|      

In [74]:
laptop_details_copy = laptop_details_copy.drop('Company','Cpu_brand','Gpu_brand','Os')
laptop_details_copy.show()

+---+------+-----+-----------+---+---+----+---+---------------+-----------------+-----------------+----------+
|Ram|Weight|Price|TouchScreen|Ips|Ppi| HDD|SSD|Company_encoded|Cpu_brand_encoded|Gpu_brand_encoded|Os_encoded|
+---+------+-----+-----------+---+---+----+---+---------------+-----------------+-----------------+----------+
|  8|     1|   11|          0|  1|227|   0|128|            7.0|              1.0|              0.0|       2.0|
|  8|     1|   11|          0|  0|128|   0|  0|            7.0|              1.0|              0.0|       2.0|
|  8|     2|   10|          0|  0|141|   0|256|            2.0|              1.0|              0.0|       1.0|
| 16|     2|   12|          0|  1|221|   0|512|            7.0|              0.0|              2.0|       2.0|
|  8|     1|   11|          0|  1|227|   0|256|            7.0|              1.0|              0.0|       2.0|
|  4|     2|   10|          0|  0|100| 500|  0|            4.0|              4.0|              2.0|       0.0|
|

In [76]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = ['Ram','Weight','TouchScreen','Ips','Ppi','HDD','SSD','Company_encoded','Cpu_brand_encoded','Gpu_brand_encoded','Os_encoded'],outputCol = 'laptop_details')
laptop_details_copy = assembler.transform(laptop_details_copy)

In [77]:
laptop_details_copy.show(truncate = False)

+---+------+-----+-----------+---+---+----+---+---------------+-----------------+-----------------+----------+--------------------------------------------------+
|Ram|Weight|Price|TouchScreen|Ips|Ppi|HDD |SSD|Company_encoded|Cpu_brand_encoded|Gpu_brand_encoded|Os_encoded|laptop_details                                    |
+---+------+-----+-----------+---+---+----+---+---------------+-----------------+-----------------+----------+--------------------------------------------------+
|8  |1     |11   |0          |1  |227|0   |128|7.0            |1.0              |0.0              |2.0       |[8.0,1.0,0.0,1.0,227.0,0.0,128.0,7.0,1.0,0.0,2.0] |
|8  |1     |11   |0          |0  |128|0   |0  |7.0            |1.0              |0.0              |2.0       |(11,[0,1,4,7,8,10],[8.0,1.0,128.0,7.0,1.0,2.0])   |
|8  |2     |10   |0          |0  |141|0   |256|2.0            |1.0              |0.0              |1.0       |[8.0,2.0,0.0,0.0,141.0,0.0,256.0,2.0,1.0,0.0,1.0] |
|16 |2     |12   |0         

In [78]:
model_inputs = laptop_details_copy.select('Price','laptop_details')
model_inputs.show(truncate = False)

+-----+--------------------------------------------------+
|Price|laptop_details                                    |
+-----+--------------------------------------------------+
|11   |[8.0,1.0,0.0,1.0,227.0,0.0,128.0,7.0,1.0,0.0,2.0] |
|11   |(11,[0,1,4,7,8,10],[8.0,1.0,128.0,7.0,1.0,2.0])   |
|10   |[8.0,2.0,0.0,0.0,141.0,0.0,256.0,2.0,1.0,0.0,1.0] |
|12   |[16.0,2.0,0.0,1.0,221.0,0.0,512.0,7.0,0.0,2.0,2.0]|
|11   |[8.0,1.0,0.0,1.0,227.0,0.0,256.0,7.0,1.0,0.0,2.0] |
|10   |[4.0,2.0,0.0,0.0,100.0,500.0,0.0,4.0,4.0,2.0,0.0] |
|12   |(11,[0,1,3,4,7,10],[16.0,2.0,1.0,221.0,7.0,2.0])  |
|11   |(11,[0,1,4,7,8,10],[8.0,1.0,128.0,7.0,1.0,2.0])   |
|11   |(11,[0,1,4,6,7,9],[16.0,1.0,157.0,512.0,3.0,1.0]) |
|11   |[8.0,2.0,0.0,1.0,157.0,0.0,256.0,4.0,1.0,0.0,0.0] |
|10   |[4.0,2.0,0.0,0.0,100.0,500.0,0.0,2.0,1.0,0.0,1.0] |
|10   |[4.0,2.0,0.0,0.0,141.0,500.0,0.0,2.0,3.0,0.0,1.0] |
|12   |[16.0,2.0,0.0,1.0,221.0,0.0,256.0,7.0,0.0,2.0,2.0]|
|10   |(11,[0,1,4,6,8,9],[4.0,2.0,141.0,256.0,3.0,2.0]) 

In [79]:
train_data,test_data = model_inputs.randomSplit([0.8,0.2],seed = 42)

In [80]:
train_data.show(truncate = False)

+-----+--------------------------------------------------+
|Price|laptop_details                                    |
+-----+--------------------------------------------------+
|9    |(11,[0,1,4,6,7,8],[4.0,1.0,112.0,32.0,2.0,2.0])   |
|9    |(11,[0,1,4,6,7,8],[4.0,1.0,157.0,32.0,9.0,2.0])   |
|9    |(11,[0,1,4,7,8],[2.0,1.0,112.0,2.0,2.0])          |
|9    |(11,[0,1,4,7,8],[2.0,1.0,112.0,12.0,2.0])         |
|9    |(11,[0,1,4,7,8],[2.0,1.0,135.0,2.0,2.0])          |
|9    |(11,[0,1,4,7,8],[2.0,1.0,135.0,3.0,2.0])          |
|9    |(11,[0,1,4,7,8],[2.0,2.0,112.0,1.0,2.0])          |
|9    |(11,[0,1,4,7,8],[4.0,1.0,112.0,1.0,2.0])          |
|9    |(11,[0,1,4,7,8],[4.0,1.0,157.0,12.0,2.0])         |
|9    |(11,[0,1,4,7,8],[4.0,1.0,166.0,12.0,2.0])         |
|9    |(11,[0,1,4,7,8],[4.0,2.0,141.0,14.0,2.0])         |
|9    |(11,[0,1,4,7,8,9],[2.0,2.0,112.0,3.0,4.0,2.0])    |
|9    |[2.0,1.0,0.0,0.0,135.0,0.0,32.0,4.0,2.0,0.0,1.0]  |
|9    |[2.0,2.0,0.0,0.0,100.0,0.0,16.0,4.0,2.0,0.0,1.0] 

In [81]:
test_data.show(truncate = False)

+-----+-------------------------------------------------+
|Price|laptop_details                                   |
+-----+-------------------------------------------------+
|9    |(11,[0,1,4,7,8],[2.0,1.0,112.0,1.0,2.0])         |
|9    |(11,[0,1,4,7,8],[2.0,1.0,135.0,3.0,2.0])         |
|9    |(11,[0,1,4,7,8],[2.0,1.0,157.0,9.0,2.0])         |
|9    |(11,[0,1,4,7,8],[4.0,2.0,141.0,14.0,2.0])        |
|9    |[4.0,2.0,0.0,0.0,100.0,500.0,0.0,3.0,2.0,0.0,1.0]|
|10   |(11,[0,1,2,4,5,8],[4.0,2.0,1.0,166.0,1000.0,3.0])|
|10   |(11,[0,1,2,4,6,8],[4.0,1.0,1.0,135.0,128.0,2.0]) |
|10   |(11,[0,1,4,5,7],[6.0,2.0,100.0,1000.0,2.0])      |
|10   |(11,[0,1,4,5,7,8],[4.0,2.0,100.0,500.0,2.0,3.0]) |
|10   |(11,[0,1,4,5,7,8],[4.0,2.0,100.0,500.0,2.0,3.0]) |
|10   |(11,[0,1,4,5,7,8],[4.0,2.0,100.0,500.0,3.0,2.0]) |
|10   |(11,[0,1,4,5,7,8],[4.0,2.0,100.0,500.0,4.0,1.0]) |
|10   |(11,[0,1,4,5,7,8],[4.0,2.0,100.0,500.0,4.0,2.0]) |
|10   |(11,[0,1,4,5,7,8],[4.0,2.0,100.0,500.0,6.0,1.0]) |
|10   |(11,[0,

**Random Forest Regrression:**

In [97]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

rf = RandomForestRegressor(featuresCol="laptop_details", labelCol="Price", numTrees=10)

# Train the model
model_rf = rf.fit(train_data)

# Make predictions
predictions_rf = model_rf.transform(test_data)

In [98]:
predictions_rf.show(truncate = False)

+-----+-------------------------------------------------+------------------+
|Price|laptop_details                                   |prediction        |
+-----+-------------------------------------------------+------------------+
|9    |(11,[0,1,4,7,8],[2.0,1.0,112.0,1.0,2.0])         |9.68737623009657  |
|9    |(11,[0,1,4,7,8],[2.0,1.0,135.0,3.0,2.0])         |9.548120910947635 |
|9    |(11,[0,1,4,7,8],[2.0,1.0,157.0,9.0,2.0])         |9.45732825261766  |
|9    |(11,[0,1,4,7,8],[4.0,2.0,141.0,14.0,2.0])        |9.85810323517357  |
|9    |[4.0,2.0,0.0,0.0,100.0,500.0,0.0,3.0,2.0,0.0,1.0]|9.98445106864245  |
|10   |(11,[0,1,2,4,5,8],[4.0,2.0,1.0,166.0,1000.0,3.0])|10.225399565011413|
|10   |(11,[0,1,2,4,6,8],[4.0,1.0,1.0,135.0,128.0,2.0]) |10.098917861709017|
|10   |(11,[0,1,4,5,7],[6.0,2.0,100.0,1000.0,2.0])      |10.589975226214005|
|10   |(11,[0,1,4,5,7,8],[4.0,2.0,100.0,500.0,2.0,3.0]) |10.092402691464422|
|10   |(11,[0,1,4,5,7,8],[4.0,2.0,100.0,500.0,2.0,3.0]) |10.092402691464422|

In [99]:
evaluator = RegressionEvaluator(labelCol="Price", predictionCol="prediction", metricName="rmse")
rmse_rf = evaluator.evaluate(predictions_rf)
print(f"Root Mean Squared Error (RMSE) for Random Forest Regressor is: {rmse_rf}")

Root Mean Squared Error (RMSE) for Random Forest Regressor is: 0.39502911663836054


In [90]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

lr = LinearRegression(featuresCol="laptop_details", labelCol="Price")

model_lr = lr.fit(train_data)

predictions_lr = model_lr.transform(test_data)

In [96]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator_lr = RegressionEvaluator(labelCol="Price", predictionCol="prediction", metricName="rmse")
rmse_lr = evaluator.evaluate(predictions_lr)
print(f"Root Mean Squared Error (RMSE) for Linear Regressor is: {rmse_lr}")

Root Mean Squared Error (RMSE) for Linear Regressor is: 0.4618880314424389


In [94]:
from pyspark.ml.regression import DecisionTreeRegressor

dt = DecisionTreeRegressor(featuresCol="laptop_details", labelCol="Price")
model_dt = dt.fit(train_data)
predictions_dt = model_dt.transform(test_data)

In [95]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator_dt = RegressionEvaluator(labelCol = "Price", predictionCol = "prediction", metricName="rmse")
rmse_dt = evaluator_dt.evaluate(predictions_dt)
print(f"Root Mean Squared Error (RMSE) for Decision Tree Regressor is: {rmse_dt}")

Root Mean Squared Error (RMSE) for Decision Tree Regressor is: 0.43672195069167075


**Conclusion:**

Based on the RMSE values, the Random Forest Regressor would be the preferred model for making predictions on dataset, as it has the lowest prediction error.